In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder,FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Örnek bir veri seti
import pandas as pd
import numpy as np


  Verisetimizi yüklüyoruz

In [3]:
path="/content/drive/MyDrive/dataset/Food_Delivery_Times.csv"
data=pd.read_csv(path)

Veri setimizi tanımak için bir fonksiyon yazıyoruz

In [4]:
def dataset_explore(df):
 print("-----------------")
 print(f"Shape: {df.shape}")
 print("-----------------")
 print(f"Types: {df.dtypes}")
 print("-----------------")
 print(f"Null Values: {df.isnull().sum()}")
 print("-----------------")
 print(f"Unique Values: {df.nunique()}")
 print("-----------------")
 print(f"Info: {df.info()}")
 print("-----------------")
 print(f"Describe: {df.describe()}")
 print("-----------------")
 print(f"Head: {df.head()}")
 print("-----------------")
 print(f"Tail: {df.tail()}")
 print("-----------------")

Fonksiyonu çalıştırıyoruz

In [5]:
dataset_explore(data)

-----------------
Shape: (1000, 9)
-----------------
Types: Order_ID                    int64
Distance_km               float64
Weather                    object
Traffic_Level              object
Time_of_Day                object
Vehicle_Type               object
Preparation_Time_min        int64
Courier_Experience_yrs    float64
Delivery_Time_min           int64
dtype: object
-----------------
Null Values: Order_ID                   0
Distance_km                0
Weather                   30
Traffic_Level             30
Time_of_Day               30
Vehicle_Type               0
Preparation_Time_min       0
Courier_Experience_yrs    30
Delivery_Time_min          0
dtype: int64
-----------------
Unique Values: Order_ID                  1000
Distance_km                785
Weather                      5
Traffic_Level                3
Time_of_Day                  4
Vehicle_Type                 3
Preparation_Time_min        25
Courier_Experience_yrs      10
Delivery_Time_min          108
dty

İşimize Yaramayacağından ötürü Order Id değişkenini veri setinden silebiliriz.

In [6]:
data=data.drop("Order_ID",axis=1)

Null değer Analizinden görüleceği üzere 4 değişkende null değerler saptandı. Bunların üçü object türüne sahip 1i float tipine sahip. Object olanların doldurulma stratejisini daha iyi belirlemek için ayrı incelemeyi deneyelim.

In [7]:
categorical_columns=data.select_dtypes(include=['object']).columns
print(data[categorical_columns])

for col in categorical_columns:
  print(data[col].unique())
print(data["Traffic_Level"].unique())

    Weather Traffic_Level Time_of_Day Vehicle_Type
0     Windy           Low   Afternoon      Scooter
1     Clear        Medium     Evening         Bike
2     Foggy           Low       Night      Scooter
3     Rainy        Medium   Afternoon      Scooter
4     Clear           Low     Morning         Bike
..      ...           ...         ...          ...
995   Clear          High     Evening          Car
996   Rainy           Low     Morning      Scooter
997   Snowy          High     Evening      Scooter
998   Clear           Low   Afternoon         Bike
999   Foggy           Low       Night      Scooter

[1000 rows x 4 columns]
['Windy' 'Clear' 'Foggy' 'Rainy' 'Snowy' nan]
['Low' 'Medium' 'High' nan]
['Afternoon' 'Evening' 'Night' 'Morning' nan]
['Scooter' 'Bike' 'Car']
['Low' 'Medium' 'High' nan]


Nasıl Bir doldurma ve normalizasyon Tekniği izleyeceğimizi belirlemek için  nümerik sütunlarımızı inceliyoruz. Hedef Değişkenimiz Olacak Olan Delivery Time Sütununu da çıkarıyoruz nümerik sütunlar arasından.

In [8]:
numerical_columns=data.select_dtypes(include=['float64','int64']).columns
numerical_columns=numerical_columns.drop("Delivery_Time_min")
print(data[numerical_columns])
print(data[numerical_columns].describe())


     Distance_km  Preparation_Time_min  Courier_Experience_yrs
0           7.93                    12                     1.0
1          16.42                    20                     2.0
2           9.52                    28                     1.0
3           7.44                     5                     1.0
4          19.03                    16                     5.0
..           ...                   ...                     ...
995         8.50                    13                     3.0
996        16.28                     8                     9.0
997        15.62                    26                     2.0
998        14.17                     8                     0.0
999         6.63                    24                     3.0

[1000 rows x 3 columns]
       Distance_km  Preparation_Time_min  Courier_Experience_yrs
count  1000.000000           1000.000000              970.000000
mean     10.059970             16.982000                4.579381
std       5.696656      

Outlierlarla ilgilenmesi için yeni bir fonksiyon yazıyoruz. Bunu function transformer ile pipeline'a entegre edeceğiz.

In [9]:
def handle_outliers(column):
    Q1 = np.percentile(column, 25)
    Q3 = np.percentile(column, 75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    column[column < lower_bound] = lower_bound
    column[column > upper_bound] = upper_bound
    return column

Yeni featurelar ekliyoruz deneme yanılma yöntemiyle faydalı olanları burada bırakıyorum.

In [10]:
def add_expected_speed(df):
    speed_mapping = {"Bike": 50, "Scooter": 15, "Car": 35}
    df["Expected_Speed"] = df["Vehicle_Type"].map(speed_mapping)
    return df

# 2. Traffic Time Impact: Traffic_Level ve Time_of_Day'a göre etki ekleme
def add_traffic_time_impact(df):
    # İlk olarak, birleşik bir sütun oluşturalım
    traffic_time_combined = df["Traffic_Level"] + "-" + df["Time_of_Day"]
    impact_mapping = {
        "Low-Morning": 1, "Low-Afternoon": 2, "Low-Evening": 3,
        "Medium-Morning": 2, "Medium-Afternoon": 3, "Medium-Evening": 4,
        "High-Morning": 3, "High-Afternoon": 4, "High-Evening": 5
    }
    df["Traffic_Time_Impact"] = traffic_time_combined.map(impact_mapping)
    return df

# 3. Weather Traffic Impact: Weather ve Traffic_Level'e göre etki ekleme
def add_weather_traffic_impact(df):
    # İlk olarak, birleşik bir sütun oluşturalım
    weather_traffic_combined = df["Weather"] + "-" + df["Traffic_Level"]
    impact_mapping = {
        "Clear-Low": 1, "Clear-Medium": 2, "Clear-High": 3,
        "Rainy-Low": 2, "Rainy-Medium": 3, "Rainy-High": 4,
        "Snowy-Low": 3, "Snowy-Medium": 4, "Snowy-High": 5,
        "Foggy-Low": 2, "Foggy-Medium": 3, "Foggy-High": 4
    }
    df["Weather_Traffic_Impact"] = weather_traffic_combined.map(impact_mapping)
    return df

# Ana fonksiyon: Tüm özellikleri eklemek için
def add_features(df):
    col_to_fill=["Weather","Traffic_Level","Time_of_Day"]
    for col in col_to_fill:
        mode_value = df[col].mode()[0]  # Access the first element of the mode Series
        df[col] = df[col].fillna(mode_value)

    df = add_weather_traffic_impact(df)
    df = add_traffic_time_impact(df)
    df = add_expected_speed(df)
    return df

add_features(data)



,Distance_km,Weather,Traffic_Level,Time_of_Day,Vehicle_Type,Preparation_Time_min,Courier_Experience_yrs,Delivery_Time_min,Weather_Traffic_Impact,Traffic_Time_Impact,Expected_Speed
0,7.93,Windy,Low,Afternoon,Scooter,12,1.0,43,NaN,2.0,15
1,16.42,Clear,Medium,Evening,Bike,20,2.0,84,2.0,4.0,50
2,9.52,Foggy,Low,Night,Scooter,28,1.0,59,2.0,NaN,15
3,7.44,Rainy,Medium,Afternoon,Scooter,5,1.0,37,3.0,3.0,15
4,19.03,Clear,Low,Morning,Bike,16,5.0,68,1.0,1.0,50
...,...,...,...,...,...,...,...,...,...,...,...
995,8.50,Clear,High,Evening,Car,13,3.0,54,3.0,5.0,35
996,16.28,Rainy,Low,Morning,Scooter,8,9.0,71,2.0,1.0,15
997,15.62,Snowy,High,Evening,Scooter,26,2.0,81,5.0,5.0,15
998,14.17,Clear,Low,Afternoon,Bike,8,0.0,55,1.0,2.0,50


Burada Preprocessing işlemleri için pipelinelarımızı hazırlıyoruz. Kategorik verilerimizi modla doldurup one hot encode uyguluyoruz. Sayısal verilerimizdeki tek eksik veriye sahip  Kuryenin deneyim yılı sayısını ortalamayla doldurmamız uygundur.  Sayısal verilerin genel olarak düzgün dağılmasından ötürü Standart Scaler kullanacağız.

In [11]:
categorical_transformer=Pipeline(steps=[('imputer',SimpleImputer(strategy="most_frequent")),
('onehot',OneHotEncoder(handle_unknown="ignore"))])

numerical_transformer=Pipeline(steps=[('imputer',SimpleImputer(strategy="mean")),
("handle_outliers",FunctionTransformer(handle_outliers,validate=False)),
 ('scaler',StandardScaler())])

Pipelinelarımızı birleştiriyoruz

In [12]:
categorical_columns=data.select_dtypes(include=['object']).columns
categorical_columns
preprocessor=ColumnTransformer(
    transformers=[('cat',categorical_transformer,categorical_columns),
 ('num',numerical_transformer,numerical_columns)])

Model için PipeLine Oluşturacağız

In [13]:
model_pipeline=Pipeline(steps=[('preprocessor',preprocessor),
 ('regressor',LinearRegression())])

Modele sokmadan önce verimizi test ve  train olmak üzere ayırıyoruz

In [14]:
features=data.drop("Delivery_Time_min",axis=1)
labels=data["Delivery_Time_min"]

X_train,X_test,y_train,y_test=train_test_split(features,labels,test_size=0.2)


Modelimizi eğitiyoruz.

In [15]:
model_pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Weather', 'Traffic_Level', 'Time_of_Day', 'Vehicle_Type'], dtype='object')),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('handle_outliers',
                                                                   FunctionTransformer(func=<function handle_outliers at 0x7e105ab777e0>)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Distance_km', 'Preparation_Time_min', 'Courier_Experience_yrs'], dtype='object'))])),
                ('regressor', LinearRegression())])

Pipeline'ımız görüldüğü üzere iki farklı veri tipini analiz ediyor. Bunlara Sırayla  encoding veyahut normalizasyon işlemleri uyguluyor eksik değerleri doldurduktan sonra.

Modelimizin perfonmasını ölçeceğiz

In [16]:
y_pred = model_pipeline.predict(X_test)

# Model performansını değerlendirme
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse=np.sqrt(mse)

model=model_pipeline.named_steps['regressor']
print(model.coef_)


print("Mean Squared Error:", mse)
print("R-squared:", r2)
print("Root Mean Squared Error:", rmse)





[-4.76938352  2.98799382  0.04181092  4.66486153 -2.92528275  6.22789248
 -5.94749309 -0.28039939  0.08332849  0.62133816 -0.10789773 -0.59676892
  0.34499848  0.48158466 -0.82658314 16.75971538  6.75789327 -1.91600241]
Mean Squared Error: 70.1993377819078
R-squared: 0.8572245123379771
Root Mean Squared Error: 8.378504507482694


Ridge yöntemini modelimizin daha genellenebilir olması için kullanacağız. Ancak skorumuzda pek değişiklik gözlenmiyor model hali hazırda overfitting'den kaçınıyor gözüküyor.

In [17]:
from sklearn.linear_model import Ridge

model_pipeline=Pipeline(steps=[('preprocessor',preprocessor),
 ('regressor',Ridge(alpha=1.5))])

model_pipeline.fit(X_train,y_train)


model=model_pipeline.named_steps['regressor']
print(model.coef_)

y_pred = model_pipeline.predict(X_test)

mse=mean_squared_error(y_test,y_pred)
r2=r2_score(y_test,y_pred)
rmse=np.sqrt(mse)


print("Mean Squared Error:", mse)
print("R-squared:", r2)
print("Root Mean Squared Error:", rmse)



[-4.74233592  2.93877659  0.05306061  4.59689197 -2.84639325  6.18016231
 -5.90239031 -0.27777199  0.07330511  0.61500724 -0.11094984 -0.57736251
  0.34336843  0.47428144 -0.81764987 16.72992158  6.74593708 -1.91352766]
Mean Squared Error: 70.24165859667042
R-squared: 0.8571384378085456
Root Mean Squared Error: 8.381029685943751


Modelimizi streamlit'de kullanmak üzere kaydediyoruz

In [18]:
import joblib


model_path="/content/drive/My Drive/model.pkl"

joblib.dump(model_pipeline, model_path)

import pickle


data_path = "/content/drive/My Drive/dataset.pkl"

with open(data_path, "wb") as f:
    pickle.dump(data, f)
